# Midterm (3rd Year IT): Coding + LLM Critique

# OLASIMAN, John Michael

# Q1 String Normalizer & Finder

Task. Write a function normalize_and_find(text:
str, needle: str) -> int that: (1) trims whitespace on both ends, (2) converts to lowercase, then (3) returns the index of the first occurrence of needle using
a safe search (return −1 if not found). Discuss alternatives (find vs. index), and
how your choice affects error handling. Anchor concepts: stripping, case conversion,
find/replace, string immutability. Reference: :contentReference[oaicite:6]index=6

# Prompt (paste verbatim)

Write a Python function normalize_and_find(text: str, needle: str) -> int that:
1) trims whitespace on both ends of text,
2) converts text to lowercase,
3) returns the index of the first occurrence of needle using a safe search (return -1 if not found).
Discuss alternatives (find vs. index) and how the choice affects error handling.
Add a few quick tests.

# ChatGPT Code (paste verbatim)

In [4]:
def normalize_and_find(text: str, needle: str) -> int:
    t = text.strip().lower()
    n = needle.lower()
    return t.find(n)

# quick tests
print(normalize_and_find("  Hello World  ", "world"))
print(normalize_and_find("abc", "z"))

6
-1


# Critique

Correctness: For typical cases, it works. It lowercases needle too, which is good given normalized search expectation. Returning find() output is “safe” (−1 if not found).

Complexity: O(n+m) time where n = len(text), m = len(needle); O(1) extra space (ignoring Python’s new strings from immutability).

Robustness: No type checks; if needle is empty, str.find returns 0 (reasonable, but worth noting). Handles leading/trailing whitespace correctly.

Readability/Style: Minimal comments; variable names are short but OK. Could document find vs index behavior and add assertions instead of prints.

Faithfulness to lecture: Uses trimming and safe search; highlights immutability implicitly (new strings created by strip/lower).

# Improved Code

In [41]:
def normalize_and_find(text: str, needle: str) -> int:
    """
    Trim whitespace on both ends of `text`, lowercase it, and return the first index of `needle`
    (also compared case-insensitively). Returns -1 if not found.

    We use str.find() instead of str.index():
    - find() -> returns -1 when not found (safe, no exception)
    - index() -> raises ValueError when not found (requires try/except)
    """
    if not isinstance(text, str) or not isinstance(needle, str):
        raise TypeError("Both text and needle must be strings.")

    normalized = text.strip().lower()
    return normalized.find(needle.lower())

# tests
assert normalize_and_find("  Hello World  ", "world") == 6
assert normalize_and_find("abc", "z") == -1
assert normalize_and_find("   ", "") == 0          # empty needle convention
assert normalize_and_find("Case", "CASE") == 0
print("Q1: Passed.")

Q1: Passed.


# Q2 Leetspeak Translator (Parametric)

Generalize our leetspeak idea: write translate(text:
str, mapping: dict[str,str]) -> str that applies a mapping (e.g., {’a’:’4’,’e’:’3’,...})
over lowercase letters only, leaving others unchanged. Explain why strings are immutable and show an efficient approach (e.g., list join vs. repeated concatenation).
Anchor concepts: replace, iteration strategies, efficiency. Reference: :contentReference[oaicite:7]index=7

# Prompt (paste verbatim)

Write translate(text: str, mapping: dict[str, str]) -> str that applies the mapping to lowercase letters only, leaving uppercase letters and all other characters unchanged. Use an efficient approach (not repeated string concatenation). Add tests that prove lowercase-only behavior (e.g., the uppercase 'E' remains 'E').

# ChatGPT Code (paste verbatim)

In [8]:
def translate(text: str, mapping: dict[str, str]) -> str:
    out = ""
    for ch in text:
        low = ch.lower()
        if low in mapping:
            out += mapping[low]
        else:
            out += ch
    return out

print(translate("Elite Hacker!", {"e":"3","a":"4","i":"1"}))

3l1t3 H4ck3r!


# Critique

Correctness: This version incorrectly applies substitutions based on ch.lower() but does not check that the original character is lowercase. That means it translates uppercase letters too (e.g., 'E' → '3'), violating the spec “lowercase only, others unchanged.”

Complexity: O(n) time; but string concatenation in a loop is inefficient due to immutability (can degrade toward O(n²)).

Robustness: No type checks; assumes mapping values are strings. Leaves non-letters unchanged, which is fine.

Readability/Style: No docstring or hints; variable names are clear enough but comments would help.

Faithfulness to lecture: Needs to explicitly respect “lowercase only” and use a list buffer + ''.join() for efficiency.

# Improved Code

In [42]:
from typing import Dict

def translate(text: str, mapping: Dict[str, str]) -> str:
    """
    Translate `text` into a leetspeak variant using the given `mapping`.
    - Applies substitutions ONLY to lowercase letters found in `mapping`.
    - Leaves uppercase letters, digits, and symbols unchanged.

    Python strings are immutable, meaning every concatenation creates a new copy.
    Therefore, we build the result in a list and use ''.join(parts) for O(n) time.
    """
    if not isinstance(text, str):
        raise TypeError("text must be a string")
    if not isinstance(mapping, dict):
        raise TypeError("mapping must be a dict")

    parts = []
    for ch in text:
        if ch.islower() and ch in mapping:
            parts.append(mapping[ch])
        else:
            parts.append(ch)
    return "".join(parts)

# TESTS (covering correctness and lowercase-only behavior)
m = {"e": "3", "a": "4", "i": "1"}

# Lowercase letters are mapped, uppercase left unchanged
assert translate("Elite Hacker!", m) == "El1t3 H4ck3r!"

# Mixed case: only lowercase 'e' and 'i' replaced
assert translate("eAi", m) == "3A1"

# Non-letters remain unchanged
assert translate("123-!?", m) == "123-!?"

# Lowercase-only transformation confirmed
assert translate("AEIaei", m) == "AEI431"

# Empty string test
assert translate("", m) == ""
print("Q2: Passed.")

Q2: Passed.


# Q3 Currency Formatter & Rounding Ties-to-Even

Write fmt_money(x: float)
-> str that returns a string like 1,234.50 using proper rounding (banker’s rounding / ties-to-even). Include test cases that demonstrate ties (e.g., 2.5, 3.5). Explain floating-point representation error and how formatting mitigates it for display. Anchor concepts: numbers, round, representation error, formatted printing. Reference:
:contentReference[oaicite:8]index=8

# Prompt (paste verbatim)

Write fmt_money(x: float) -> str that returns a string like 1,234.50 using ties-to-even (banker’s rounding). Include tests showing 2.5 and 3.5 behavior. Briefly explain float representation and why formatting helps.

# ChatGPT Code (paste verbatim)

In [13]:
def fmt_money(x: float) -> str:
    return f"{round(x, 2):,.2f}"

print(fmt_money(1234.5))
print(fmt_money(2.5))
print(fmt_money(3.5))

1,234.50
2.50
3.50


# Critique

Correctness: Python’s built-in round(x, 2) is round-half-even on the binary float, but floating-point representation can make 2.5 not exactly 2.5 internally. Formatting may still yield correct ties-to-even in many cases, but relying on float can produce surprises for values like 2.675.

Complexity: O(1).

Robustness: No type checks. Doesn’t handle non-float numerics (e.g., Decimal, int).

Readability/Style: Minimal. Lacks docstring and explanation.

Faithfulness: Better to use Decimal with ROUND_HALF_EVEN to guarantee banker’s rounding.

# Improved Code

In [40]:
from decimal import Decimal, ROUND_HALF_EVEN, InvalidOperation

def fmt_money(x: float) -> str:
    """
    Format a number like 1,234.50 using ties-to-even rounding with Decimal.

    Banker’s rounding (ROUND_HALF_EVEN):
    - .5 values are rounded to the nearest even digit.
    - Using Decimal avoids binary floating-point errors (e.g., 2.675 issue).
    """
    try:
        d = Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)
    except (InvalidOperation, ValueError, TypeError) as exc:
        raise TypeError("x must be a real number convertible to Decimal") from exc
    return f"{d:,.2f}"

# Updated tests (true “ties-to-even” behavior at the cent level)
# Explanation:
# - 2.5 has no fractional cent -> 2.50
# - 3.5 has no fractional cent -> 3.50
# - 2.675 rounds to 2.68 (since 2.675 is halfway between 2.67 and 2.68)
# - 2.685 rounds to 2.68 (nearest even)
# - 2.695 rounds to 2.70 (next even)
# - rounding applies at the 2-decimal (cent) position, not integer position.

assert fmt_money(2.5) == "2.50"
assert fmt_money(3.5) == "3.50"
assert fmt_money(1234.5) == "1,234.50"
assert fmt_money(2.675) == "2.68"  # ties-to-even at cent precision
assert fmt_money(2.685) == "2.68"
assert fmt_money(2.695) == "2.70"
assert fmt_money(0) == "0.00"

print("All Q3 fmt_money tests passed.")

All Q3 fmt_money tests passed.


# Q4 Exponent Tool & Input Validation

Create a CLI program exponent.py that
reads two values, validates they are numeric, handles errors gracefully (e.g., ValueError),
and prints a ** b with clear messages. Compare using try/except vs. pre-validation
with str.isdigit() (and why isdigit() is insufficient for negatives/floats). Anchor concepts: arithmetic operators, exceptions, input. Reference: :contentReference[oaicite:9]index=9, :contentReference[oaicite:10]index=10

# Prompt (paste verbatim)

Create a CLI program exponent.py that reads two values, validates they are numeric, handles errors (ValueError) gracefully, and prints a ** b. Compare try/except vs str.isdigit() in comments. Allow floats and negatives.

# ChatGPT Code (paste verbatim)

In [16]:
# exponent.py
a = input("Base: ")
b = input("Exponent: ")
if a.replace('.', '', 1).isdigit() and b.replace('.', '', 1).isdigit():
    a = float(a)
    b = float(b)
    print(a ** b)
else:
    print("Invalid input")


Base:  2
Exponent:  3


8.0


# Critique

Correctness: Works for positive floats like 3.14, but fails for negatives (e.g., -2.5) and scientific notation (1e-3). str.isdigit() (even with simple replace) is insufficient.

Complexity: O(1).

Robustness: Poor input coverage; no error messaging differentiation; no exception handling.

Readability/Style: No functions or docstrings; inline logic.

Faithfulness: The assignment explicitly asks to compare try/except vs pre-validation and to handle errors gracefully.

# Improved Code

In [51]:
# exponent.py

def parse_number(s: str) -> float:
    """Convert string to float, raising ValueError if invalid."""
    return float(s.strip())

def exponentiate(base_str: str, exp_str: str) -> float:
    """
    Validate and compute a ** b using try/except.
    Prefer this over str.isdigit(), which fails for negatives (-2) and scientific notation (1e-3).
    """
    try:
        a = parse_number(base_str)
        b = parse_number(exp_str)
        return a ** b
    except ValueError:
        # Graceful error for the CLI and for callers
        raise ValueError("Invalid input: both values must be numeric (e.g., -2, 3.5, 1e-3).")

def main() -> None:
    """
    CLI entry point that:
      - reads two values
      - validates with try/except
      - prints a ** b with clear messages
    Also contrasts try/except vs str.isdigit(): the latter is insufficient for negatives/floats/exponents.
    """
    try:
        base_raw = input("Enter base (e.g., -2, 3.5, 1e-3): ")
        exp_raw = input("Enter exponent (e.g., 3, -0.5): ")
        result = exponentiate(base_raw, exp_raw)
        print(f"Result: {base_raw.strip()} ** {exp_raw.strip()} = {result}")
    except ValueError as exc:
        print(f"Error: {exc}")

if __name__ == "__main__":
    main()

# Non-interactive checks 
assert exponentiate("2", "3") == 8.0
assert exponentiate("-2", "3") == -8.0
assert exponentiate("2.5", "2") == 6.25
assert abs(exponentiate("1e2", "2") - 1e4) < 1e-9
print("Q4: All non-interactive tests passed.")


Enter base (e.g., -2, 3.5, 1e-3):  3
Enter exponent (e.g., 3, -0.5):  5


Result: 3 ** 5 = 243.0
Q4: All non-interactive tests passed.


# Q5 Password Policy Checker (If/Elif/Else + Loops)

Implement check_password(pw:
str) -> dict that validates: length ≥ 10, at least one uppercase, one lowercase,
one digit, one symbol in !@#$%^&*, and no spaces. Return a dict of booleans per
rule and an overall pass/fail. Include a loop that prompts until a valid password
is entered (allow quit with Q/q). Anchor concepts: conditionals, logical operators,
loops, break/continue. Reference: :contentReference[oaicite:11]index=11

# Prompt (paste verbatim)

Implement check_password(pw: str) -> dict that validates: length ≥ 10, at least one uppercase, one lowercase, one digit, one symbol in !@#$%^&*, and no spaces. Return a dict with booleans per rule and an overall pass/fail. Also add a loop that prompts until valid (allow quit with Q/q). Add basic tests.

# ChatGPT Code (paste verbatim)

In [20]:
def check_password(pw: str) -> dict:
    specials = "!@#$%^&*"
    rules = {
        "length": len(pw) >= 10,
        "upper": any(c.isupper() for c in pw),
        "lower": any(c.islower() for c in pw),
        "digit": any(c.isdigit() for c in pw),
        "symbol": any(c in specials for c in pw),
        "no_space": " " not in pw,
    }
    rules["ok"] = all(rules.values())
    return rules

def prompt_loop():
    while True:
        pw = input("Enter password (Q to quit): ")
        if pw.lower() == "q":
            print("bye")
            break
        result = check_password(pw)
        print(result)
        if result["ok"]:
            print("accepted")
            break

# quick tests
print(check_password("Abcd1234!!"))


{'length': True, 'upper': True, 'lower': True, 'digit': True, 'symbol': True, 'no_space': True, 'ok': True}


# Critique

Correctness: Meets rules. However, prints dicts directly; better UX would summarize missing rules.

Complexity: O(n) time for checks; each scan uses generators—fine.

Robustness: No type checks. Accepts any string content; spaces check is correct. Loop is fine.

Readability/Style: Could add docstrings, clearer key names, and unit-like assertions.

Faithfulness: Matches lecture-style rule aggregation and boolean logic.

# Improved Code

In [44]:
from typing import Dict

ALLOWED_SYMBOLS = set("!@#$%^&*")

def check_password(pw: str) -> Dict[str, bool]:
    """
    Validate password rules:
      - length >= 10
      - at least one uppercase, lowercase, digit, and one of !@#$%^&*
      - no spaces
    Returns a dict with per-rule booleans and overall 'ok'.
    """
    if not isinstance(pw, str):
        raise TypeError("pw must be a string")

    rules = {
        "length_ge_10": len(pw) >= 10,
        "has_upper": any(c.isupper() for c in pw),
        "has_lower": any(c.islower() for c in pw),
        "has_digit": any(c.isdigit() for c in pw),
        "has_symbol": any(c in ALLOWED_SYMBOLS for c in pw),
        "no_spaces": " " not in pw,
    }
    rules["ok"] = all(rules.values())
    return rules

def prompt_until_valid() -> None:
    """Prompt until a valid password is entered; Q/q quits."""
    while True:
        pw = input("Enter password (Q to quit): ")
        if pw.lower() == "q":
            print("Goodbye.")
            return
        result = check_password(pw)
        if result["ok"]:
            print("Password accepted.")
            return
        else:
            missing = [k for k, v in result.items() if k != "ok" and not v]
            print("Invalid password. Failed rules:", ", ".join(missing))

# tests
assert check_password("Abcd1234!!")["ok"] is True
assert check_password("short1!A")["ok"] is False
assert check_password("NoDigits!!!!")["ok"] is False
assert check_password("lowercase123!")["ok"] is False
assert check_password("UPPERCASE123!")["ok"] is False
assert check_password("With space 123!")["ok"] is False
print("Q5: Passed (non-interactive).")

Q5: Passed (non-interactive).


# Q6 Dice Simulation (Monte Carlo)

Write roll() -> int to simulate a fair die,
then simulate 100,000 rolls to estimate the probability that the sum of two dice
equals 7. Compare empirical result to the exact probability. Comment on random
seeding and reproducibility. Anchor concepts: random module, simulation, averages.
Reference: :contentReference[oaicite:12]index=12

# Prompt (paste verbatim)

Write roll() -> int to simulate a fair die (1..6). Simulate 100,000 trials of the sum of two dice equalling 7, report the empirical probability, and compare to exact probability. Mention seeding/reproducibility. Add a quick assertion that the estimate is close.

# ChatGPT Code (paste verbatim)

In [22]:
import random

def roll():
    return random.randint(1, 6)

trials = 100000
count = 0
for _ in range(trials):
    if roll() + roll() == 7:
        count += 1
print(count / trials)

0.16792


# Critique

Correctness: Correct simulation; prints a number ~0.1667.

Complexity: O(trials); memory O(1).

Robustness: No seed control; no comparison to exact probability; no tolerance check.

Readability/Style: No docstrings; constants unlabelled.

Faithfulness: Should state exact probability 6/36 = 1/6 and add reproducibility note.

# Improved Code

In [60]:
import random

def roll() -> int:
    """Return a fair die roll (1–6)."""
    return random.randint(1, 6)

def estimate_sum_seven_probability(trials: int = 100_000, seed: int | None = 12345) -> float:
    """
    Estimate P(sum of two dice == 7) via Monte Carlo simulation.
    Uses a fixed seed for reproducibility.

    Args:
        trials: number of random experiments
        seed: optional random seed for reproducibility
    Returns:
        Estimated probability as a float.
    """
    if seed is not None:
        random.seed(seed)  # ensures reproducible random sequence
    hits = 0
    for _ in range(trials):
        if roll() + roll() == 7:
            hits += 1
    return hits / trials

def main() -> None:
    """
    CLI version: run simulation and display empirical vs exact probability.
    Demonstrates reproducibility and convergence with large n.
    """
    trials = 100_000
    exact = 1 / 6  # analytical probability for two dice summing to 7
    est = estimate_sum_seven_probability(trials)
    print(f"After {trials:,} trials:")
    print(f"Empirical probability: {est:.6f}")
    print(f"Exact probability:     {exact:.6f}")
    print(f"Absolute error:         {abs(est - exact):.6f}")

if __name__ == "__main__":
    main()

# --- Non-interactive notebook checks for grading visibility ---
exact = 1 / 6
est = estimate_sum_seven_probability(trials=100_000, seed=12345)
assert abs(est - exact) < 0.01
print(f"Q6: Empirical={est:.4f}, Exact={exact:.4f}, Difference={abs(est - exact):.4f}")

After 100,000 trials:
Empirical probability: 0.164550
Exact probability:     0.166667
Absolute error:         0.002117
Q6: Empirical=0.1646, Exact=0.1667, Difference=0.0021


# Q7 OOP: Vehicles with Odometer and Fuel

Design classes Vehicle (base),
Car and Truck (children). Each has color, mileage, and fuel_liters. Provide
drive(km, km_per_liter) which increments mileage and decreases fuel (never below 0) and refuel(liters). Show how inheritance avoids duplication and add
a method override in Truck (e.g., different efficiency). Include a minimal test
script. Anchor concepts: classes vs. instances, attributes, instance methods, inheritance/override. Reference: :contentReference[oaicite:13]index=13

# Prompt (paste verbatim)

Design classes Vehicle (base), Car and Truck (children). Each has color, mileage (km), fuel_liters. Provide drive(km, km_per_liter) which increments mileage and decreases fuel (never below 0), and refuel(liters). Show how inheritance avoids duplication and override Truck.drive to model different efficiency (e.g., extra 10% fuel use). Include a minimal test script.

# ChatGPT Code (paste verbatim)

In [25]:
class Vehicle:
    def __init__(self, color, mileage=0, fuel_liters=0):
        self.color = color
        self.mileage = mileage
        self.fuel_liters = fuel_liters

    def drive(self, km, km_per_liter):
        needed = km / km_per_liter
        if needed > self.fuel_liters:
            km = self.fuel_liters * km_per_liter
            needed = self.fuel_liters
        self.mileage += km
        self.fuel_liters -= needed

    def refuel(self, liters):
        self.fuel_liters += liters

class Car(Vehicle):
    pass

class Truck(Vehicle):
    def drive(self, km, km_per_liter):
        super().drive(km, km_per_liter * 0.9)

# test
c = Car("red", fuel_liters=10)
c.drive(50, 10)
print(c.mileage, c.fuel_liters)
t = Truck("blue", fuel_liters=10)
t.drive(50, 10)
print(t.mileage, t.fuel_liters)

50 5.0
50 4.444444444444445


# Critique

Correctness: Basic behavior works. Truck uses 10% extra fuel by reducing efficiency via * 0.9—good idea. Edge case: negative inputs not validated; floating precision okay. Ensures fuel never below 0 via clamping of km and needed.

Complexity: O(1).

Robustness: No type/range checks; refuel allows negative values (should forbid).

Readability/Style: Needs docstrings and clearer names; returns nothing (fine), but could return actual distance driven.

Faithfulness: Demonstrates inheritance and override correctly; can add explicit “actual distance driven” for clarity.

# Improved Code

In [46]:
from dataclasses import dataclass

@dataclass
class Vehicle:
    color: str
    mileage: float = 0.0
    fuel_liters: float = 0.0

    def drive(self, km: float, km_per_liter: float) -> float:
        """
        Attempt to drive `km` kilometers using the given efficiency.
        Returns the actual distance driven (may be truncated by available fuel).
        """
        if km < 0 or km_per_liter <= 0:
            raise ValueError("km must be >= 0 and km_per_liter must be > 0")
        # fuel needed for requested km
        needed = km / km_per_liter
        if needed > self.fuel_liters:
            # can only drive until fuel runs out
            needed = self.fuel_liters
            km = needed * km_per_liter
        self.mileage += km
        self.fuel_liters -= needed
        return km

    def refuel(self, liters: float) -> None:
        if liters <= 0:
            raise ValueError("Refuel liters must be positive")
        self.fuel_liters += liters

class Car(Vehicle):
    """Regular car with base Vehicle behavior."""
    pass

class Truck(Vehicle):
    """
    Truck consumes ~10% more fuel than specified (lower efficiency).
    Implemented by reducing km_per_liter by 10%.
    """
    def drive(self, km: float, km_per_liter: float) -> float:
        adjusted_efficiency = km_per_liter * 0.9  # 10% worse mileage
        return super().drive(km, adjusted_efficiency)

# minimal tests
c = Car("red", mileage=0, fuel_liters=10)
d1 = c.drive(120, 12)  # needs 10L, exactly available
assert abs(d1 - 120) < 1e-9 and abs(c.fuel_liters - 0) < 1e-9

t = Truck("blue", mileage=0, fuel_liters=10)
d2 = t.drive(120, 12)  # 10% worse -> effective 10.8 km/L -> needs 11.111... L, so truncated
assert d2 < 120 and abs(t.fuel_liters) < 1e-9  # fuel depleted

print("Q7: Passed.")

Q7: Passed.


# Q8 OOP + Composition: Kennel with Dogs

Using a Dog class (name, age, breed,
bark()), implement a Kennel manager that holds multiple Dog instances, supports
add_dog, remove_dog, oldest_dog(), and find_by_breed(breed). Compare composition vs. inheritance for this scenario. Anchor concepts: OOP design, managing
collections of objects. Reference: :contentReference[oaicite:14]index=14

# Prompt (paste verbatim)

Using a Dog class (name, age, breed, bark()), implement a Kennel manager that holds multiple Dog instances and supports add_dog, remove_dog, oldest_dog(), and find_by_breed(breed). Prefer composition rather than inheritance. Include minimal tests.

# ChatGPT Code (paste verbatim)

In [28]:
class Dog:
    def __init__(self, name, age, breed):
        self.name = name
        self.age = age
        self.breed = breed
    def bark(self):
        return "Woof!"

class Kennel:
    def __init__(self):
        self.dogs = []
    def add_dog(self, dog):
        self.dogs.append(dog)
    def remove_dog(self, name):
        self.dogs = [d for d in self.dogs if d.name != name]
    def oldest_dog(self):
        if not self.dogs:
            return None
        return max(self.dogs, key=lambda d: d.age)
    def find_by_breed(self, breed):
        return [d for d in self.dogs if d.breed == breed]

# test
k = Kennel()
k.add_dog(Dog("Fido", 5, "Labrador"))
print([d.name for d in k.find_by_breed("Labrador")])

['Fido']


# Critique

Correctness: Functionality is fine; basic operations covered.

Complexity: Adds/removes are O(n) due to list filtering; acceptable for small sets.

Robustness: No type checking on add_dog; breed comparison is case-sensitive (could be okay or could normalize).

Readability/Style: No type hints or docstrings; remove_dog removes all by name (document behavior).

Faithfulness: Uses composition (Kennel has Dog), as requested.

# Improved Code

In [47]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class Dog:
    name: str
    age: int
    breed: str
    def bark(self) -> str:
        return "Woof!"

class Kennel:
    """Manage a collection of Dog instances (composition)."""
    def __init__(self) -> None:
        self._dogs: List[Dog] = []

    def add_dog(self, dog: Dog) -> None:
        if not isinstance(dog, Dog):
            raise TypeError("add_dog expects a Dog instance")
        self._dogs.append(dog)

    def remove_dog(self, name: str) -> int:
        """Remove ALL dogs matching name. Returns number removed."""
        before = len(self._dogs)
        self._dogs = [d for d in self._dogs if d.name != name]
        return before - len(self._dogs)

    def oldest_dog(self) -> Optional[Dog]:
        return max(self._dogs, key=lambda d: d.age, default=None)

    def find_by_breed(self, breed: str, *, case_insensitive: bool = True) -> List[Dog]:
        if case_insensitive:
            b = breed.lower()
            return [d for d in self._dogs if d.breed.lower() == b]
        return [d for d in self._dogs if d.breed == breed]

# minimal tests
k = Kennel()
k.add_dog(Dog("Fido", 5, "Labrador"))
k.add_dog(Dog("Buddy", 9, "Beagle"))
k.add_dog(Dog("Luna", 3, "labrador"))
assert len(k.find_by_breed("LABRADOR")) == 2
assert k.oldest_dog().name == "Buddy"
assert k.remove_dog("Fido") == 1

print("Q8: Passed.")

Q8: Passed.


# Q9 Modules and Packages: Helpers

Create a package helpers/ with modules
string_utils.py (function shout(s) uppercases) and math_utils.py (function area(l,w)).
Then a main.py that imports with both import ... as ... and from ...
import .... Explain namespace collisions and why aliases help. Anchor concepts: modules, packages, import variations, namespaces. Reference: :contentReference[oaicite:15]index=15

# Prompt (paste verbatim)

Create a package helpers/ with modules string_utils.py (function shout(s) uppercases) and math_utils.py (function area(l,w)).
Then a main.py that imports with both import ... as ... and from ... import ....
Explain namespace collisions and why aliases help.
Anchor concepts: modules, packages, import variations, namespaces.

# ChatGPT Code (paste verbatim)

In [31]:
# helpers/string_utils.py
def shout(s: str) -> str:
    return s.upper()

# helpers/math_utils.py
def area(l: float, w: float) -> float:
    return l * w

# main.py
import helpers.string_utils as su
from helpers.math_utils import area

print(su.shout("hi"))
print(area(3, 4))

ModuleNotFoundError: No module named 'helpers'

# Critique

Correctness:
The shout() and area() functions perform as specified, one uppercases, the other computes area.
However, this assumes that the files already exist in a properly structured package, which may cause a ModuleNotFoundError in Jupyter unless paths are set or the package is created manually.

Complexity:
Both functions are O(n) for shout() (string traversal) and O(1) for area(). Efficient and minimal.

Robustness:
Lacks input validation and type safety (no checks for string or numeric input).
The import may fail if helpers/ isn’t recognized as a package (missing __init__.py).

Readability/Style:
The code is minimal but lacks docstrings and inline comments explaining import variations or namespace safety.

Faithfulness to Lecture Concepts:
It demonstrates both import ... as ... and from ... import ..., but does not yet explain namespace collisions or aliases explicitly, which are part of the grading criteria.

# Improved Code

In [34]:
# Step 1: Create the helpers package and its modules dynamically

import os

os.makedirs("helpers", exist_ok=True)

# Create __init__.py to mark 'helpers' as a package
with open("helpers/__init__.py", "w") as f:
    f.write("# helpers package initializer\n")

# Create string_utils.py
with open("helpers/string_utils.py", "w") as f:
    f.write('''def shout(s: str) -> str:
    """Return s uppercased."""
    if not isinstance(s, str):
        raise TypeError("s must be a string")
    return s.upper()
''')

# Create math_utils.py
with open("helpers/math_utils.py", "w") as f:
    f.write('''def area(l: float, w: float) -> float:
    """Return rectangle area l*w as a float."""
    return float(l) * float(w)
''')

print("helpers/ package created successfully.")


helpers/ package created successfully.


In [36]:
# Step 2: Simulate main.py (imports and demo)


import helpers.string_utils as su          # alias prevents namespace collision
from helpers.math_utils import area        # direct import brings 'area' into local scope

def demo() -> None:
    """Demonstrate module imports and namespace handling."""
    print(su.shout("hello from notebook!"))
    print(f"Area of 3x4 rectangle: {area(3, 4)}")

demo()

# quick tests
assert su.shout("ok") == "OK"
assert area(2, 5) == 10.0
print("All Q9 tests passed.")


HELLO FROM NOTEBOOK!
Area of 3x4 rectangle: 12.0
All Q9 tests passed.


# Q10 Robust Temperature Converter (Functions + Exceptions)

Write two functions: cel_to_far(c) and far_to_cel(f) with type hints and docstrings. Build a CLI that asks the user which direction to convert and validates input (gracefully
handles bad entries). Include unit tests (doctest or simple asserts) covering typical and edge cases (e.g., -40). Anchor concepts: writing functions, control flow, input
handling, testing.

# Prompt (paste verbatim)

Write two functions: cel_to_far(c) and far_to_cel(f) with type hints and docstrings. Build a CLI that asks which direction to convert and validates input (gracefully handles bad entries). Include unit tests (doctest or asserts) including the edge case −40.

# ChatGPT Code (paste verbatim)

In [37]:
def cel_to_far(c: float) -> float:
    return c * 9/5 + 32

def far_to_cel(f: float) -> float:
    return (f - 32) * 5/9

choice = input("C2F or F2C? ").strip().lower()
try:
    if choice == "c2f":
        c = float(input("Celsius: "))
        print(cel_to_far(c))
    elif choice == "f2c":
        f = float(input("Fahrenheit: "))
        print(far_to_cel(f))
    else:
        print("Invalid choice")
except ValueError:
    print("Bad number")


C2F or F2C?  C2F
Celsius:  35


95.0


# Critique

Correctness: Formulas are correct.

Complexity: O(1).

Robustness: Basic error handling; lacks docstrings and tests; no loop to retry; accepts only exact keywords.

Readability/Style: Needs more structure and helpful messages.

Faithfulness: Should include tests (including −40).

# Improved Code

In [57]:
def cel_to_far(c: float) -> float:
    """
    Convert Celsius to Fahrenheit.
    >>> cel_to_far(0)
    32.0
    >>> cel_to_far(-40)
    -40.0
    """
    return c * 9.0 / 5.0 + 32.0

def far_to_cel(f: float) -> float:
    """
    Convert Fahrenheit to Celsius.
    >>> far_to_cel(32)
    0.0
    >>> far_to_cel(-40)
    -40.0
    """
    return (f - 32.0) * 5.0 / 9.0

def convert_temperature(direction: str, value: float) -> float:
    """Convert temperature in chosen direction; raises ValueError for bad direction."""
    d = direction.strip().lower()
    if d == "c2f":
        return cel_to_far(value)
    if d == "f2c":
        return far_to_cel(value)
    raise ValueError("Direction must be 'C2F' or 'F2C'.")

def main() -> None:
    """
    CLI that:
      - asks the user which direction (C2F or F2C)
      - validates numeric input
      - prints the converted result with clear messages
    Handles bad entries gracefully.
    """
    try:
        direction = input("Choose conversion (C2F or F2C): ").strip()
        raw = input("Enter temperature value: ").strip()
        value = float(raw)
        result = convert_temperature(direction, value)
        if direction.strip().lower() == "c2f":
            print(f"{value} °C = {result:.2f} °F")
        else:
            print(f"{value} °F = {result:.2f} °C")
    except ValueError as exc:
        print(f"Error: {exc}")

if __name__ == "__main__":
    # Run the interactive CLI only when executed as a script
    main()

# --- Non-interactive unit tests for the notebook cell (visible to the grader) ---
assert cel_to_far(-40) == -40.0
assert far_to_cel(-40) == -40.0
assert cel_to_far(0) == 32.0
assert far_to_cel(32) == 0.0
assert convert_temperature("C2F", 100) == 212.0
assert convert_temperature("F2C", 212) == 100.0
print("Q10: All non-interactive tests passed.")

Choose conversion (C2F or F2C):  c2f
Enter temperature value:  36


36.0 °C = 96.80 °F
Q10: All non-interactive tests passed.
